In [3]:
import numpy as np
import random
import pandas as pd

In [ ]:
istanbul_metro = {
    "M2": [
        "YENIKAPI", "VEZNECILER", "HALIC", "SISHANE", "TAKSIM",
        "OSMANBEY", "SISLI", "GAYRETTEPE", "LEVENT", "4. LEVENT",
        "SANAYI MAH.", "SEYRANTEPE", "ITU - AYAZAGA", "ATATURK OTO SANAYI", "DARUSSAFAKA",
        "HACIOSMAN"
    ],
    "M3": [
        "BAKIRKOY IDO", "OZGURLUK MEYDANI", "INCIRLI", "HAZNEDAR", "ILKYUVA",
        "MOLLA GURANI", "KIRAZLI", "YENIMAHALLE", "MAHMUTBEY", "ISTOC",
        "IKITELLI SANAYI", "TURGUT OZAL", "SITELER", "BASAK KONUTLARI", "METROKENT",
        "ONURKENT", "SEHIR HASTANESI", "TOPLU KONUTLAR", "KAYASEHIR MERKEZ"
],
    "M4": [
        "KADIKOY", "AYRILIKCESME", "ACIBADEM", "UNALAN", "GOZTEPE",
        "YENISAHRA", "KOZYATAGI", "BOSTANCI", "KUCUKYALI", "MALTEPE",
        "HUZUREVI", "GULSUYU", "ESENKENT CEVIZLI", "HASTANE ADLIYE", "SOGANLIK",
        "KARTAL", "YAKACIK", "PENDIK", "TAVSANTEPE", "HASTANE",
        "YAYALAR", "KURTKOY", "SABIHA GOKCEN HAVALIMANI",
    ],
    "M5": [
        "USKUDAR", "FISTIKAGACI", "BAGLARBASI", "ALTUNIZADE", "KISIKLI",
        "BULGURLU", "UMRANIYE", "CARSI", "YAMANEVLER", "CAKMAK",
        "IHLAMURKUYU", "ALTINSEHIR", "IMAM HATIP LISESI", "DUDULLU", "NECIP FAZIL",
        "CEKMEKOY", "MECLIS", "SARIGAZI", "SANCAKTEPE", "SAMANDIRA MERKEZ"
    ],
    "M6": [
        "LEVENT  KONKORS", "NISPETIYE", "ETILER", "BOGAZICI"
    ],
    "M7": [
        "M YILDIZ", "M FULYA", "MECIDIYEKOY", "CAGLAYAN", "KAGITHANE",
        "NURTEPE", "ALIBEYKOY", "CIRCIR", "VEYSELKARANI", "YESILPINAR",
        "KAZIMKARABEKIR", "YENIMAHALLE", "KARADENIZ MAH.", "TEKSTILKENT", "ORUCREIS",
        "GOZTEPE", "MAHMUTBEY M HOL"
    ],
    "M8": [
        "BOSTANCI", "EMIN ALI PASA", "AYSE KADIN", "ICERENKOY", "KUCUKBAKKALKOY", "KAYISDAGI",
        "MEVLANA", "IMES", "MODOKO - KEYAP", "DUDULLU", "HUZUR", "PARSELLER"
    ],
    "M9": [
        "ATAKOY", "YENIBOSNA", "COBANCESME", "29 EKIM CUMHURIYET", "DOGU SANAYI", "MIMAR SINAN",
        "15 TEMMUZ", "HALKALI CADDESI", "ATATURK MAHALLESI", "BAHARIYE", "MASKO", 
        "IKITELLI SANAYI", "ZIYA GOKALP MAH.", "OLIMPIYAT"
    ],
    "M11": [
        "GAYRETTEPE", "KAGITHANE", "HASDAL", "KEMERBURGAZ", "GOKTURK", "IHSANIYE",
        "ISTANBUL HAVALIMANI", "KARGO TERMINALI", "TASOLUK", "ARNAVUTKOY HASTANE"
    ],

    "T5": [
        "EMINONU", "KUCUKPAZAR", "CIBALI", "FENER", "BALAT",
        "AYVANSARAY", "FESHANE", "EYUP TELEFERIK", "EYUP DEVLET HASTANESI", "SILAHTARAGA",
        "UNIVERSITE", "ALIBEYKOY", "ALIBEYKOY METRO", "CEP OTOGAR"],

    "T4": [
        "TOPKAPI", "FETIHKAPI", "VATAN", "EDIRNEKAPI", "SEHITLIK",
        "DEMIRKAPI", "TOPCULAR", "RAMI", "BEREC", "SAGMALCILAR",
        "CUKURCESME", "ALI FUAT BASGIL", "TASKOPRU", "KARADENIZ MAHALLESI", "T4 KIPTAS VENEZIA",
        "CUMHURIYET", "BASTABYA", "HACI SUKRU", "YENIMAHALLE", "SULTANCIFTLIGI",
        "CEBECI", "MESCIDI SELAM"],

    "T1": [ 
        "KABATAS", "FINDIKLI", "TOPHANE", "KARAKOY", "EMINONU",
        "SIRKECI", "GULHANE", "SULTANAHMET", "CEMBERLITAS", "BEYAZIT",
        "LALELI", "AKSARAY", "YUSUFPASA", "HASEKI", "FINDIKZADE",
        "CAPA", "PAZARTEKKE", "TOPKAPI", "ATATURK OGRENCI YURDU", "MERKEZEFENDI",
        "AKSEMSETTIN", "MITHATPASA", "ZEYTINBURNU", "MEHMETAKIF", "...",
        "GUNGOREN", "AKINCILAR", "SOGANLI", "...", "GUNESTEPE",
        "BAGCILAR"]
}


M2:  ['YENIKAPI', 'VEZNECILER', 'HALIC', 'SISHANE', 'TAKSIM', 'OSMANBEY', 'SISLI', 'GAYRETTEPE', 'LEVENT', '4. LEVENT', 'SANAYI MAHALLESI', 'SEYRANTEPE', 'İTÜ - AYAZAĞA', 'ATATÜRK OTO SANAYI', 'DARÜŞŞAFAKA', 'HACIOSMAN']


NameError: name 'array' is not defined

In [ ]:
data = pd.read_csv(r'data/hourly_transportation_202410.csv')
data = data[data["transport_type_id"] == 2]
data = data[["transition_date", "transition_hour", "line_name", "number_of_passage", "station_poi_desc_cd"]]
data = data.rename(columns={"transition_date": "date", "transition_hour": "hour", "line_name": "line", "number_of_passage": "passenger_count", "station_poi_desc_cd": "station"})
data = data[data["hour"] >= 6]

# Remove unwanted words (KUZEY, GUNEY, DOGU, BATI) and numbers from the 'station' column
data['station'] = data['station'].str.replace(r'\b(KUZEY|GUNEY|DOGU|BATI)\b', '', regex=True)
# Remove all digits and numbers
data['station'] = data['station'].str.replace(r'\d+', '', regex=True)
data['station'] = data['station'].str.replace(r'[()\-/]', '', regex=True)  # Remove symbols: (), -, /
data['station'] = data['station'].str.strip()
data["station"] = data["station"].astype(str)

data = data.groupby(['date', 'hour', 'line', 'station'], as_index=False)['passenger_count'].sum()





array(['ATATURK OTO SANAYI', 'DARUSSAFAKA', 'GAYRETTEPE', 'HACIOSMAN',
       'HALIC', 'ITU', 'LEVENT', 'OSMANBEY', 'SANAYI MAH.', 'SEYRANTEPE',
       'SEYRANTEPE  STAD GIRISI', 'SISHANE', 'SISLI', 'TAKSIM',
       'VEZNECILER', 'YENIKAPI'], dtype=object)

In [13]:
data[data["line"] == "M7"].station.unique()

array(['ALIBEYKOY', 'CAGLAYAN', 'CIRCIR', 'GOZTEPE', 'KAGITHANE',
       'KARADENIZ MAH.', 'KAZIMKARABEKIR', 'M FULYA', 'M YILDIZ',
       'MAHMUTBEY M HOL', 'MECIDIYEKOY', 'NURTEPE', 'ORUCREIS',
       'TEKSTILKENT', 'VEYSELKARANI', 'YENIMAHALLE', 'YESILPINAR'],
      dtype=object)

In [19]:
# calculate the avarage passengers  for every line and station based on differnet dates than drop dates column
def calculate_average_passengers(data):
    # Create a new DataFrame to store the average passengers
    average_data = pd.DataFrame(columns=['line', 'station', 'average_passenger_count'])

    # Iterate through each line and station
    for line in data['line'].unique():
        for station in data['station'].unique():
            # Filter the data for the current line and station
            filtered_data = data[(data['line'] == line) & (data['station'] == station)]
            
            # Calculate the average passenger count
            if not filtered_data.empty:
                average_passenger_count = filtered_data['passenger_count'].mean()
                average_data = average_data.append({'line': line, 'station': station, 'average_passenger_count': average_passenger_count}, ignore_index=True)

    return average_data
# Calculate the average passengers
average_data = calculate_average_passengers(data)
average_data

AttributeError: 'DataFrame' object has no attribute 'append'

In [18]:
data[data["line"] == "M2"].station.unique()

array(['ATATURK OTO SANAYI', 'DARUSSAFAKA', 'GAYRETTEPE', 'HACIOSMAN',
       'HALIC', 'ITU', 'LEVENT', 'OSMANBEY', 'SANAYI MAH.', 'SEYRANTEPE',
       'SEYRANTEPE  STAD GIRISI', 'SISHANE', 'SISLI', 'TAKSIM',
       'VEZNECILER', 'YENIKAPI'], dtype=object)

In [ ]:
dict = {}
for i in range(len(data)):
    if data["line"][i] not in dict:
        dict[data["line"][i]] = {}
    if data["date"][i] not in dict[data["line"][i]]:
        dict[data["line"][i]][data["date"][i]] = {}
    if data["hour"][i] not in dict[data["line"][i]][data["date"][i]]:
        dict[data["line"][i]][data["date"][i]][data["hour"][i]] = {}
    if data["station"][i] not in dict[data["line"][i]][data["date"][i]][data["hour"][i]]:
        dict[data["line"][i]][data["date"][i]][data["hour"][i]][data["station"][i]] = {}
    dict[data["line"][i]][data["date"][i]][data["hour"][i]][data["station"][i]] = data["passenger_count"][i]

In [ ]:
NUM_LINES = 5  # example number of metro lines
POPULATION_SIZE = 50
OPERATIONAL_COST = 1000
GENERATIONS = 100
FREQ_BOUNDS = (2, 10) 
STOP_TIME_BOUNDS = (20, 90)  # Seconds

w1, w2, w3, w4 = 1.0, 3.0, 1.5, 1.2

In [ ]:
def cost_function(frequency, stop_time, line_data):
    op_cost = 0

    for date in line_data:
        for hour in line_data[date]:
            op_cost += frequency * OPERATIONAL_COST
            for station in line_data[date][hour]:
                station_passenger = line_data[date][hour][station][direction_1]
                metro_passenger = line_data[date][hour][station][cum_direction_1]

                wait_cost = station_passenger * frequency/2 + stop_time + metro_passenger * stop_time
                unserved_cost = (station_passenger - (max_capacity - cum_direction_1)) * unserved_cost 
                # cum_direction_1 is number of passengers when train reaches the station number of available places for ht new passengers is calculated as
                # max_capacity - cum_direction_1 and the remaining passengers are unserved
                density_cost = cum_direction_1 * density_cost 
                # because the cum_direction_1 is the number of passengers in the train harm of the density affects until the next station
                standing_cost = (cum_direction_1 - max__seat_capacity) * standing_cost

    return op_cost + wait_cost + unserved_cost + density_cost + standing_cost


In [98]:
def initialize_population():
    population = []
    for _ in range(POPULATION_SIZE):
        frequencies = [random.randint(*FREQ_BOUNDS) for _ in range(NUM_LINES)]
        stop_times = [random.randint(*STOP_TIME_BOUNDS) for _ in range(NUM_LINES)]
        population.append((frequencies, stop_times))
    return population


In [99]:
def select_population(population, fitnesses):
    sorted_pop = [x for _, x in sorted(zip(fitnesses, population))]
    return sorted_pop[:POPULATION_SIZE // 2]


In [100]:
def crossover(parents):
    offspring = []
    while len(offspring) < POPULATION_SIZE:
        p1, p2 = random.sample(parents, 2)
        point = random.randint(1, NUM_LINES - 1)
        new_freq = p1[0][:point] + p2[0][point:]
        new_stop = p1[1][:point] + p2[1][point:]
        offspring.append((new_freq, new_stop))
    return offspring

In [101]:
def mutate(individual, mutation_rate=0.1):
    for i in range(NUM_LINES):
        if random.random() < mutation_rate:
            individual[0][i] = random.randint(*FREQ_BOUNDS)
        if random.random() < mutation_rate:
            individual[1][i] = random.randint(*STOP_TIME_BOUNDS)
    return individual

In [103]:
def run_genetic_algorithm(passenger_data):
    population = initialize_population()
    for generation in range(GENERATIONS):
        fitnesses = [cost_function(freqs, stops, passenger_data) for freqs, stops in population]
        best = min(fitnesses)
        print(f"Generation {generation} - Best Cost: {best}")
        selected = select_population(population, fitnesses)
        offspring = crossover(selected)
        population = [mutate(ind) for ind in offspring]
    return population[np.argmin(fitnesses)]


In [ ]:
mock_passenger_data = None  # Replace with real preprocessed data

In [106]:
best_solution = run_genetic_algorithm(mock_passenger_data)
print("Best Solution:", best_solution)

Generation 0 - Best Cost: 0
Generation 1 - Best Cost: 0
Generation 2 - Best Cost: 0
Generation 3 - Best Cost: 0
Generation 4 - Best Cost: 0
Generation 5 - Best Cost: 0
Generation 6 - Best Cost: 0
Generation 7 - Best Cost: 0
Generation 8 - Best Cost: 0
Generation 9 - Best Cost: 0
Generation 10 - Best Cost: 0
Generation 11 - Best Cost: 0
Generation 12 - Best Cost: 0
Generation 13 - Best Cost: 0
Generation 14 - Best Cost: 0
Generation 15 - Best Cost: 0
Generation 16 - Best Cost: 0
Generation 17 - Best Cost: 0
Generation 18 - Best Cost: 0
Generation 19 - Best Cost: 0
Generation 20 - Best Cost: 0
Generation 21 - Best Cost: 0
Generation 22 - Best Cost: 0
Generation 23 - Best Cost: 0
Generation 24 - Best Cost: 0
Generation 25 - Best Cost: 0
Generation 26 - Best Cost: 0
Generation 27 - Best Cost: 0
Generation 28 - Best Cost: 0
Generation 29 - Best Cost: 0
Generation 30 - Best Cost: 0
Generation 31 - Best Cost: 0
Generation 32 - Best Cost: 0
Generation 33 - Best Cost: 0
Generation 34 - Best Cos